These are the main methods for the **LSH algorithm**. One sets up the main data structure, the LSH Cache. And the other computes candidate duplicates.

In [ ]:
import itertools
import json
from lsh import cache, minhash # https://github.com/mattilyra/lsh

# Each bin = band = use of a hash function. bins contain buckets. 
# Each bucket = group of similar text according to bin's hash fn
def candidate_duplicates(lshcache):
    candidate_pairs = set()
    for b in lshcache.bins:
        for bucket_id in b:
            if len(b[bucket_id]) > 1:
                pairs_ = set(itertools.combinations(b[bucket_id], r=2))
                candidate_pairs.update(pairs_)
    return candidate_pairs

def create_cache(sstubs, char_ngram=5, seeds=100, bands=5, hashbytes=4):
    hasher = minhash.MinHasher(seeds=seeds, char_ngram=char_ngram, hashbytes=hashbytes)
    if seeds % bands != 0:
        raise ValueError('Seeds has to be a multiple of bands. {} % {} != 0'.format(seeds, bands))
    
    lshcache = cache.Cache(num_bands=bands, hasher=hasher)
    for sstub_id, sstub in enumerate(sstubs):
        code = sstub["sourceBeforeFix"]
        lshcache.add_fingerprint(hasher.fingerprint(code), doc_id=sstub_id)
    return lshcache
    


This is script runs the LSH algorithm on our SStuBs dataset. It also produces a list of candidate duplicates.

In [ ]:
with open("modified-dataset.json", 'r') as f:
    sstubs = json.load(f)
    print("Total number of SStuBs: ", len(sstubs))

lshcache = create_cache(sstubs, char_ngram=5, seeds=100, bands=1, hashbytes=4)
candidate_pairs = candidate_duplicates(lshcache)


Total number of SStuBs:  29057


The final script groups SStuBs into buckets. It adds attribute "bucketHash" to each SStuB elements and writes the new dataset into file "sstubs-0104-bucket-hash.json"

In [ ]:
# We will only look at clone groups in the first bin. 
# Threat to validity: False negative rate might be high since we did not go through all bins
first_bin = lshcache.bins[0]

print("Number of buckets: ", len(first_bin))

for bucket_id in first_bin:
    for sstub_index in first_bin[bucket_id]:
        sstubs[sstub_index]["bucketHash"] = bucket_id

with open("sstubs-0104-bucket-hash.json", "w") as f:
    json.dump(sstubs, f, sort_keys=True)


with open("sstubs-0104-grouped.json", "w") as f:
    sstub_map = {}
    for bucket_id in first_bin:
        bucket = []
        for sstub_index in first_bin[bucket_id]:
            bucket.append(sstubs[sstub_index])
        sstub_map[bucket_id] = bucket

    json.dump(sstub_map, f, sort_keys=True)

Number of buckets:  13439


Random code snippet for analysing random stuff. Currently total # of SStuBs and projects.

In [ ]:
import json
with open("modified-dataset.json", "r") as f:
    s = set()
    sstubs = json.load(f)
    print("SStuBs in total: ", len(sstubs))
    for stub in sstubs:
        s.add(stub["projectName"])
    print("Number of projects in total: ", len(s))

SStuBs in total:  29057
Number of projects in total:  410
